# 7번 문제

### 0. DB 커넥션
+ config/db_config.py 파일에서 아래 정보 수정 필요
+ host, dbname, user, password

In [12]:
# 주석제거 후 설치 필요
#!pip install psycopg

In [13]:
import imp
import psycopg2
import pandas as pd
import config.db_config as config # db_config 접속 정보 / 테스트 시 config/db_config.py 수정 필요
imp.reload(config)  # 수정 시 reload 

psql_config = config.psql_config

conn = psycopg2.connect(host=psql_config['host'],  # end-point 정보
                        dbname=psql_config['dbname'],  # db 명
                        user=psql_config['user'],  # 계정
                        password=psql_config["password"],  # 패스워드
                        port=psql_config["port"]  # 포트
                       )

cur = conn.cursor()  # cursor 생성

### 1. 테이블 4종 생성
- 1. person 테이블
- 2. visit_occurrence 테이블
- 3. drug_exposure 테이블
- 4. condition_occurrence 테이블

In [14]:
# 쿼리파일 read
f = open("./query/ddl/7_1.sql", "r", encoding="utf-8")
query = f.read()
f.close()
print(query[:100])

/*
 * 7번 문제
 * 
 * 테이블 생성 DDL 문
 * 내용 :
 *  테이블 및 시퀀스 drop
 *  0. 랜덤번호 부여를 위한 pseudo_encrypt() 함수 생성


In [15]:
cur.execute(query)

### 2. note 데이터 파싱
- 모두 동일한 사람으로 추정 (생년월일, 성별, 인종 같음)
- 각 정보들의 index 번호 확인
    - Race: 2
    - Ethnicity: 3
    - Gender: 4
    - Age: 5
    - Birth Date: 6
    - 내원일자, 기관명 : 13
    - 내원타입 : 14

In [16]:
from collections import defaultdict

cur.execute("SELECT * FROM clinical_note")
note_data = [x[0] for x in cur.fetchall()]

In [17]:
person_info = set()  # persion info 저장
visit_info = defaultdict(list)  # person별 visit info 저장 

for i, _  in enumerate(note_data):
#     print("# {note_num} note".format(note_num=i+1))  # 확인용
    for idx, x in enumerate(note_data[i].split("\n")):
#         print(idx, x)  # 정보 확인용 
        if idx == 2:
            tmp_race = x.split()[1]
        elif idx == 3:
            tmp_eth = x.split()[1]
        elif idx == 4:
            tmp_gender = x.split()[1]
        elif idx == 5:
            tmp_age = x.split()[1]
        elif idx == 6:
            tmp_birth = x.split(":")[1].strip()
        elif idx == 13:
            tmp_visit_date = x.split(":")[0].strip()
            tmp_visit_site = x.split(":")[1].strip()
        elif idx == 14:
            tmp_visit_type = x.split(":")[1].strip()
        else:
            pass
        
    tmp_person = (tmp_race, tmp_eth, tmp_gender, tmp_age, tmp_birth)  # person 정보
    tmp_visit = (tmp_visit_date, tmp_visit_site, tmp_visit_type)  # visit 정보

    person_info.add(tmp_person)  # person 추가
    visit_info[tmp_person].append(tmp_visit)  # person별 visit 정보 추가

### 3. 결과

In [18]:
from pprint import pprint

#### 아래 주석 제거하면 출력 가능 

In [19]:
# pprint(person_info)
# pprint(visit_info[person_info.pop()])

### 4. 추가 필요 절차
1. note에서 정보를 파싱하는 룰 추가
2. DB에 인서트